In [12]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, SGD, Nadam, RMSprop
import cv2
import sys
import matplotlib.pyplot as plt
import os

In [13]:
# Load weights of convolution layers
from keras import backend as K
vgg_model = VGGFace(include_top=False,weights='vggface', input_shape=(200,200,3))
print('CNN Weights Loaded.')

CNN Weights Loaded.


In [14]:
# Training FC layers while freezing remaining CNN layers
nb_class = 10
hidden_dim = 512

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

# Finetuning: only the last 3 fully connected layers are trained. The rest are made non-trainable.
layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count = layer_count+1
for l in range(layer_count-3):
    custom_vgg_model.layers[l].trainable=False
    
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 200, 200, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 200, 200, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 100, 100, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 100, 100, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 50, 50, 128)       0         
__________

In [15]:
# Loading train, test and validation data
train_data_dir = '/Users/seba/desktop/NewDataset/Train'
validation_data_dir = '/Users/seba/desktop/NewDataset/Validate'
#test_data_dir = '/Users/seba/Desktop/YaleDataset/test'
nb_train_samples = 210
nb_validation_samples = 45
nb_test_samples = 45 # per class
img_height = 200
img_width = 200 
batch_size = 32

In [16]:
MEAN_VALUE = np.array([103.939, 116.779, 123.68])   # BGR
def preprocess(img):
    # img is (channels, height, width), values are 0-255
    img = img[::-1]  # switch to BGR
    img -= MEAN_VALUE
    return img

In [17]:
# prepare data augmentation configuration
validate_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,  
    featurewise_center=True,
    featurewise_std_normalization=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,
    featurewise_center=True, 
    featurewise_std_normalization=True,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2100 images belonging to 10 classes.
Found 450 images belonging to 10 classes.


In [18]:
weights_path = '/Users/seba/desktop/Weights/weights_14.h5'
log_path = '/Users/seba/Desktop/Logs/log_14/'
checkpointer = ModelCheckpoint(filepath=weights_path, verbose=1,save_best_only = True, monitor = 'val_loss')

In [19]:
# load best weights 
#custom_vgg_model.load_weights(weights_path)

# Compile model
custom_vgg_model.compile(optimizer=SGD(lr=0.01, momentum=0.7, decay=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,patience=0, min_lr=1e-6)
tensorboard_callback=TensorBoard(log_dir= log_path, histogram_freq=0,write_graph=True,write_images=False)


In [20]:
# Finetuning: train last 3 fully connected layers
custom_vgg_model.fit_generator( 
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size,callbacks=[checkpointer,tensorboard_callback])

Epoch 1/50
6/6 [==============================] - 223s - loss: 2.3012 - acc: 0.0885 - val_loss: 2.2859 - val_acc: 0.2188
Epoch 2/50
6/6 [==============================] - 218s - loss: 2.2999 - acc: 0.1302 - val_loss: 2.2892 - val_acc: 0.1875
Epoch 3/50
6/6 [==============================] - 208s - loss: 2.3020 - acc: 0.1302 - val_loss: 2.2859 - val_acc: 0.2500
Epoch 4/50
6/6 [==============================] - 212s - loss: 2.2902 - acc: 0.2240 - val_loss: 2.2732 - val_acc: 0.5000
Epoch 5/50
6/6 [==============================] - 181s - loss: 2.2886 - acc: 0.2292 - val_loss: 2.2425 - val_acc: 0.5000
Epoch 6/50
6/6 [==============================] - 238s - loss: 2.2821 - acc: 0.2552 - val_loss: 2.2593 - val_acc: 0.5000
Epoch 7/50
6/6 [==============================] - 209s - loss: 2.2810 - acc: 0.3125 - val_loss: 2.2624 - val_acc: 0.0000e+00
Epoch 8/50
6/6 [==============================] - 216s - loss: 2.2764 - acc: 0.3594 - val_loss: 2.2488 - val_acc: 0.4062
Epoch 9/50
6/6 [============

In [34]:
res = dict((v,k) for k,v in train_generator.class_indices.items())
res1 = np.array(res)
labelmap = np.expand_dims(res1, axis=0)
labelmap[0]

labelmap[0]

{0: 'Farida',
 1: 'Hazem',
 2: 'Heba',
 3: 'Ibrahim',
 4: 'Maryam',
 5: 'Nesma',
 6: 'Seba',
 7: 'Shady',
 8: 'Tarek',
 9: 'Youssef'}

In [23]:
import keras.utils
from keras import utils as np_utils

x=[1,1,]
x = np.asarray(x)
x = np.expand_dims(x, axis=0)
x.shape
y_test = keras.utils.to_categorical(np.random.randint(10, size=(1, 1)), num_classes=10)
y_test[0][:]=1

print y_test

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


In [40]:
label=-1
y_test = keras.utils.to_categorical(np.random.randint(10, size=(1, 1)), num_classes=10)

avgscore_perclass=[]
avgscore_total=[]

for dir in os.listdir('/Users/seba/desktop/NewDataset/Test/'):
    if (dir!='.DS_Store'):
        label=label+1
        y_test[0][:]=0   #initialize all labels to zero
        y_test[0][label]=1  # current label
 
        for fn in os.listdir('/Users/seba/desktop/NewDataset/Test/'+dir):
            if fn.endswith(".jpg"):
                img = cv2.imread('/Users/seba/desktop/NewDataset/test/'+dir+'/'+fn,1) # read as RGB 
                img = cv2.resize(img, (200,200))
                x = np.asarray(img, dtype='float32')
                x = preprocess(x)
                # rescaling
                x= x* 1./255
                x = np.expand_dims(x, axis=0)
                #print x.shape
                # preprocessing on input data
                score = custom_vgg_model.evaluate(x,y_test,verbose=0)
                #print (labelmap[0][label], score[1]*100) 
                avgscore_perclass.append(score[1] * 100)
                avgscore_total.append(score[1] * 100)

        print(labelmap[0][label], "%s: %.2f%%" % (custom_vgg_model.metrics_names[1], np.mean(avgscore_perclass)))
        avgscore_perclass=[] # reset to zero after each class 
print ('Total Average Accuracy: ', np.mean(avgscore_total))

('Farida', 'acc: 100.00%')
('Hazem', 'acc: 100.00%')
('Heba', 'acc: 91.11%')
('Ibrahim', 'acc: 93.33%')
('Maryam', 'acc: 84.44%')
('Nesma', 'acc: 100.00%')
('Seba', 'acc: 97.78%')
('Shady', 'acc: 100.00%')
('Tarek', 'acc: 77.78%')
('Youssef', 'acc: 100.00%')
('Total Average Score: ', 94.444444444444443)


In [ ]:
label=-1
y_test = keras.utils.to_categorical(np.random.randint(10, size=(1, 1)), num_classes=10)

avgscore=[]
for dir in os.listdir('/Users/seba/desktop/NewDataset/Test/'):
    if (dir!='.DS_Store'):
        label=label+1
        y_test[0][:]=0   #initialize all labels to zero
        y_test[0][label]=1  # current label
 
        for fn in os.listdir('/Users/seba/desktop/NewDataset/Test/'+dir):
            if fn.endswith(".jpg"):
                img = cv2.imread('/Users/seba/desktop/NewDataset/test/'+dir+'/'+fn,1) # read as RGB 
                img = cv2.resize(img, (200,200))
                x = np.asarray(img, dtype='float32')
                x = preprocess(x)
                # rescaling
                x= x* 1./255
                x = np.expand_dims(x, axis=0)
                #print x.shape
                #print fn
                # preprocessing on input data
                score = custom_vgg_model.evaluate(x,y_test,verbose=0)
                #print score[1]*100
                avgscore.append(score[1] * 100)
        print(labelmap[0][label], "%s: %.2f%%" % (custom_vgg_model.metrics_names[1], score[1]*100))

In [ ]:
# Train last FC layer or train whole network including CNN layers? 
# Validation_steps??
# Subtract mean of images?? 


# for fluctuations in accuracy, you can: use dropout, increase batch size, reduce learning rate

#The number of samples processed for each epoch is batch_size * steps_per_epochs.
#steps_per_epoch = number of times weights are updated per epoch